# Deep Recurrent Learning (N-BEATS) vs. Traditional Machine Learning
## Advanced Machine Learning in Finance - Individual Project

The aim of this project is to compare the performance of two machine learning models, N-Beats and XGBoost, for predicting sales figures. The challenge is to use available data that has a correlation with the sales of various companies, such as credit card transactions and geographical data, to predict the quarterly sales figures.

The goal is to compare the accuracy of predictions for the current quarter, as well as the next couple of quarters, between N-Beats and XGBoost.

N-Beats is a deep learning model designed specifically for time-series forecasting, while XGBoost is a popular tree-based model used for various types of predictive modeling. This project aim to give insights into comparative forecasting results from these two models and their applications in the field of sales prediction.

The outcome of this project will be a comparison between the two models, including a discussion of the results and their implications for businesses and organizations looking to improve their financial forecasting capabilities.

In [28]:
import darts
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch 
import os
import sys
import sklearn 
import torch
from utils import Processor
from darts.dataprocessing.transformers import (
    Scaler,
    MissingValuesFiller,
    Mapper,
    InvertibleMapper,
)
from darts import TimeSeries
import darts.metrics as metrics
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler

In [29]:
model = Processor('exabel_data')

In [30]:
model.files

['revenue.xlsx', '.DS_Store', 'covid_us.csv', 'spend_amounts_aggregated.xlsx']

In [31]:
revenue = model.create_df(model.revenue)
spendings = model.create_df(model.spendings)
df = model.merge_spendings_revenue(spendings, revenue)
df = model.create_stationary_covariates(df)
df = model.add_holiday(df)
df = model.add_war_to_df(df)
df = model.encode_index(df)

In [27]:
df.head()

,mic,ticker,time,nw_total_sales_a_total,nw_total_sales_b_total,Sales_Actual_fiscal,Sales_Estimate_fiscal,year,month,quarter,is_holiday,is_weekend,is_workday,war
0,0,FLWS,2019-03-31,54356.41,NaN,248.413,243.11867,2019,3,1,0,1,0,0
1,0,FLWS,2019-06-30,54514.94,NaN,259.398,255.14700,2019,6,2,0,1,0,0
2,0,FLWS,2019-12-29,41047.13,NaN,605.642,610.46940,2019,12,4,0,1,0,0
3,0,FLWS,2020-03-29,60962.10,NaN,278.776,266.56613,2020,3,1,0,1,0,0
4,0,FLWS,2020-06-28,58101.76,33479.59,417.956,372.81314,2020,6,2,0,1,0,0


In [17]:
set(df['mic'])
encoding = {'XAMS' :  0, 'XLON' : 1, 'XMEX' : 2, 'XNAS' : 3, 'XNYS' : 4, 'XPAR' : 5, 'XTKS' : 6, 'XTSE'  : 7, 'NaN' : 8}

{'XAMS', 'XLON', 'XMEX', 'XNAS', 'XNYS', 'XPAR', 'XTKS', 'XTSE', nan}

In [22]:
# def impute_missing(df):
#     df.sort_values('time')
#     df.set_index('date')
#     df['sales_imputed'] = df['sales'].rolling(window=4, min_periods=1).mean()
#     df['sales_imputed'].fillna(method='ffill', inplace=True)
#     return df

df = encode_exchange_index(df)

In [55]:
df.head(10)

,mic,ticker,time,nw_total_sales_a_total,nw_total_sales_b_total,Sales_Actual_fiscal,Sales_Estimate_fiscal,year,month,quarter,date,is_holiday,is_weekend,is_workday
0,XNAS,FLWS,2019-03-31,54356.41,NaN,248.413,243.11867,2019,3,1,2019-03-31,0,1,0
1,XNAS,FLWS,2019-06-30,54514.94,NaN,259.398,255.14700,2019,6,2,2019-06-30,0,1,0
2,XNAS,FLWS,2019-12-29,41047.13,NaN,605.642,610.46940,2019,12,4,2019-12-29,0,1,0
3,XNAS,FLWS,2020-03-29,60962.10,NaN,278.776,266.56613,2020,3,1,2020-03-29,0,1,0
4,XNAS,FLWS,2020-06-28,58101.76,33479.59,417.956,372.81314,2020,6,2,2020-06-28,0,1,0
5,XNAS,FLWS,2020-09-27,NaN,16974.41,283.772,266.05200,2020,9,3,2020-09-27,0,1,0
6,XNAS,FLWS,2020-12-27,27247.28,19563.16,877.256,755.28217,2020,12,4,2020-12-27,0,1,0
7,XNAS,FLWS,2021-03-28,34826.96,30724.94,474.234,412.80344,2021,3,1,2021-03-28,0,1,0
8,XNAS,FLWS,2021-06-27,39150.59,29499.70,486.983,474.08414,2021,6,2,2021-06-27,0,1,0
9,XNAS,FLWS,2021-09-26,52785.60,NaN,309.373,295.86444,2021,9,3,2021-09-26,0,1,0


In [58]:
# Group the data by the 'ticker' column
grouped = df.groupby('ticker')
grouped.head(10)
grouped = df.isna()

In [ ]:
test_a.tail(30)

In [ ]:
series = model.create_darts_series_from_df(df)
series = model.series_fill_missing_vals(series)
series_train, series_val = model.series_train_test(series=series)

In [ ]:
model = NBEATSModel(input_chunk_length=24, output_chunk_length=12)
model.fit(series_train)

In [ ]:
preds = model.predict(len(series_val), series=series);
preds.plot(label='preds');
series_val.plot(label='ground truth');
plt.legend()

In [ ]:
preds

In [ ]:
preds = fill_missing_vals(preds)

In [ ]:
metrics.mape(actual_series=series_val, pred_series=preds)